In [33]:

import numpy as np
import pandas as pd

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [35]:
df = pd.read_csv('train.csv')

In [36]:

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [38]:

df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [39]:
# Step 1 -> train/test/split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=0.2,
                                                random_state=42)

In [40]:

X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [41]:

y_train.sample(5)

458    1
882    0
825    0
389    1
440    1
Name: Survived, dtype: int64

In [43]:
# imputation transformer

trf1=ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')


In [48]:
# one hot encoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_outputbut =False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [56]:
##extra....ohe er por dataframe dekhar jonno

X_encoded_df = pd.DataFrame(
    trf2.fit_transform(X_train),
    columns = list(trf2.named_transformers_['ohe_sex_embarked'].get_feature_names_out(['Sex', 'Embarked'])) +
              [col for i, col in enumerate(X_train.columns) if i not in [1, 6]]
)

In [55]:
X_encoded_df

,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Pclass,Age,SibSp,Parch,Fare
0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,45.5,0.0,0.0,28.5000
1,0.0,1.0,0.0,0.0,1.0,0.0,2.0,23.0,0.0,0.0,13.0000
2,0.0,1.0,0.0,0.0,1.0,0.0,3.0,32.0,0.0,0.0,7.9250
3,0.0,1.0,0.0,0.0,1.0,0.0,3.0,26.0,1.0,0.0,7.8542
4,1.0,0.0,0.0,0.0,1.0,0.0,3.0,6.0,4.0,2.0,31.2750
...,...,...,...,...,...,...,...,...,...,...,...
707,1.0,0.0,0.0,0.0,1.0,0.0,3.0,21.0,0.0,0.0,7.6500
708,0.0,1.0,0.0,0.0,1.0,0.0,1.0,NaN,0.0,0.0,31.0000
709,0.0,1.0,0.0,0.0,1.0,0.0,3.0,41.0,2.0,0.0,14.1083
710,1.0,0.0,0.0,0.0,1.0,0.0,1.0,14.0,1.0,2.0,120.0000


In [50]:
# Scaling
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [57]:
# Feature selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [58]:
# train the model
trf5 = DecisionTreeClassifier()


## **Create Pipeline**

In [59]:
pipe=Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)])

## **using make_pipeline**

In [62]:
'''
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer

trf1 = make_column_transformer(
    (SimpleImputer(), [2]),  # Age এর জন্য ডিফল্ট ইমপিউটেশন (mean)
    (SimpleImputer(strategy='most_frequent'), [6]),  # Embarked এর জন্য most frequent fill
    remainder='passthrough'
)
'''
'''
# Alternate Syntax creating make_pipeline
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)
'''


"\nfrom sklearn.impute import SimpleImputer\nfrom sklearn.compose import make_column_transformer\n\ntrf1 = make_column_transformer(\n    (SimpleImputer(), [2]),  # Age এর জন্য ডিফল্ট ইমপিউটেশন (mean)\n    (SimpleImputer(strategy='most_frequent'), [6]),  # Embarked এর জন্য most frequent fill\n    remainder='passthrough'\n)\n"

In [63]:
# train
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x0000017077E7D080>)),
                ('trf5', DecisionTreeClassifier())])

## **Explore the Pipeline**

In [64]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x0000017077E7D080>),
 'trf5': DecisionTreeClassifier()}

In [72]:
# Display Pipeline

from sklearn import set_config
set_config(display='diagram')

In [69]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([29.49884615])

In [71]:
# Display Pipeline

from sklearn import set_config
set_config(display='diagram')

In [73]:

# Predict
y_pred = pipe.predict(X_test)

In [74]:
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0], dtype=int64)

In [75]:

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

## **Cross Validation using Pipeline**

In [76]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.6391214419383433

In [77]:
# gridsearchcv
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [78]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x0000017077E7D080>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

## **Exporting the Pipeline**


In [79]:

# export 
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))
